In [0]:
import pandas as pd
import numpy as np
import csv
import os
import seaborn as sns
import matplotlib.pyplot as plt 
import sklearn as sk
from google.colab import drive
from xgboost import XGBClassifier, XGBRegressor
from sklearn.datasets import load_iris
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, confusion_matrix, mean_squared_error
from timeit import default_timer as timer

In [0]:
drive.mount('/Downloads/')

Drive already mounted at /Downloads/; to attempt to forcibly remount, call drive.mount("/Downloads/", force_remount=True).


In [0]:
train = pd.read_csv('train_no_outlier.csv')

In [0]:
train.describe()

,id,caller_sum_6,record_sum_6,time_sum_6,caller_sum_4,record_sum_4,time_sum_4,caller_sum_5,record_sum_5,time_sum_5,brand,city_flag,day,caller_sum_45,caller_sum_456,record_sum_45,record_sum_456,time_sum_45,time_sum_456,flag
count,5.342453e+06,5.202918e+06,5.202918e+06,5.202918e+06,5.342453e+06,5.342453e+06,5.342453e+06,5.111871e+06,5.111871e+06,5.111871e+06,5.342453e+06,5.338391e+06,5.342439e+06,5.111871e+06,5.093941e+06,5.111871e+06,5.093941e+06,5.111871e+06,5.093941e+06,5.342453e+06
mean,2.644639e+07,3.027962e+01,9.050052e+01,3.977738e+02,2.916872e+01,9.360663e+01,4.151043e+02,3.219321e+01,9.776337e+01,4.313617e+02,2.217962e+00,6.306114e-01,1.412181e+03,6.228491e+01,9.333892e+01,1.947144e+02,2.876912e+02,8.605591e+02,1.269172e+03,1.594943e-01
std,1.389389e+07,3.564153e+01,9.710813e+01,4.544947e+02,3.677139e+01,1.031859e+02,4.873672e+02,3.804231e+01,1.033142e+02,4.887626e+02,6.446771e-01,4.832529e-01,1.250285e+03,7.280402e+01,1.082334e+02,2.030889e+02,2.995577e+02,9.461679e+02,1.392849e+03,3.661365e-01
min,5.130834e+06,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,-1.000000e+00,6.100000e+01,2.000000e+00,3.000000e+00,2.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.287921e+07,1.100000e+01,3.000000e+01,9.900000e+01,1.000000e+01,2.900000e+01,9.600000e+01,1.200000e+01,3.300000e+01,1.110000e+02,2.000000e+00,0.000000e+00,3.230000e+02,2.300000e+01,3.500000e+01,6.800000e+01,1.010000e+02,2.360000e+02,3.510000e+02,0.000000e+00
50%,2.426377e+07,2.100000e+01,6.400000e+01,2.540000e+02,2.000000e+01,6.600000e+01,2.580000e+02,2.200000e+01,7.000000e+01,2.770000e+02,2.000000e+00,1.000000e+00,1.055000e+03,4.300000e+01,6.500000e+01,1.400000e+02,2.060000e+02,5.620000e+02,8.310000e+02,0.000000e+00
75%,3.925556e+07,3.800000e+01,1.170000e+02,5.310000e+02,3.600000e+01,1.210000e+02,5.550000e+02,4.000000e+01,1.260000e+02,5.750000e+02,3.000000e+00,1.000000e+00,2.241000e+03,7.700000e+01,1.150000e+02,2.490000e+02,3.680000e+02,1.147000e+03,1.690000e+03,0.000000e+00
max,4.882184e+07,6.676000e+03,7.083000e+03,3.037500e+04,8.350000e+03,1.095400e+04,2.171200e+04,9.001000e+03,9.208000e+03,2.983200e+04,3.000000e+00,1.000000e+00,8.946000e+03,1.735100e+04,2.402700e+04,1.915500e+04,2.623800e+04,4.061700e+04,5.802900e+04,1.000000e+00


In [0]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# splitting X and Y from train dataset
X_train = np.array(train.drop(['id','open_date', 'caller_sum_45', 'caller_sum_456',
                              'record_sum_45', 'record_sum_456', 'time_sum_45', 'time_sum_456', 'flag'], axis = 1))
Y_train = np.array(train['flag'])

X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

weight2 = 0.8
weight1 = 0.2

start = timer()
xgb_model = XGBClassifier(scale_pos_weight = weight2/weight1)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
end = timer()

print("time to run the model: ", end-start)
print(" * Accuracy score for the model: ", xgb_model.score(X_test, y_test))
print(" * Confusion matrix for the model: ", confusion_matrix(y_test, y_pred))

('time to run the model: ', 431.50917196273804)
(' * Accuracy score for the model: ', 0.8079852801754999)
(' * Confusion matrix for the model: ', array([[746698, 151493],
       [ 53673, 116627]]))


In [0]:
print('the ROC score is:',roc_auc_score(y_test, y_pred))

('the ROC score is:', 0.7580840384619029)
